In [1]:
import firebase_admin
from firebase_admin import credentials, firestore
import pandas as pd

def initialize_firestore(service_account_key_path):
    cred = credentials.Certificate(service_account_key_path)

    firebase_admin.initialize_app(cred)
    return firestore.client()

def fetch_top_readings(db, collection_name, order_by_field, limit=10):
    query = (
        db.collection(collection_name)
        .order_by(order_by_field, direction=firestore.Query.DESCENDING)
        .limit(limit)
    )
    results = query.stream()
    data = [doc.to_dict() for doc in results]
    return data

C:\Users\praty\AppData\Roaming\Python\Python311\site-packages\pandas\core\arrays\masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (


In [ ]:
service_account_key_path = ""
db = initialize_firestore(service_account_key_path)

In [3]:
collection_name = "readings3"
order_by_field = "timestamp"
data = fetch_top_readings(db, collection_name, order_by_field)

df = pd.DataFrame(data)
desired_order = ["timestamp", "ax", "ay", "az", "gx", "gy", "gz", "mx", "my", "mz", "Latitude", "Longitude"]
df = df[desired_order]
df

,timestamp,ax,ay,az,gx,gy,gz,mx,my,mz,Latitude,Longitude
0,2024-11-27 12:07:22.641368+00:00,0.614711,6.123769,7.642292,0.056992,-0.021838,0.032624,-40.856251,-6.431250,2.212500,13.556612,80.025009
1,2024-11-26 04:53:34.718018+00:00,0.054468,4.452617,8.825027,-0.070041,0.160056,0.026232,-40.143753,15.562501,5.343750,13.556643,80.025030
2,2024-11-25 05:46:08.153831+00:00,0.145448,0.018555,9.883265,0.000133,-0.000266,0.000133,-53.362503,27.637501,-8.662500,13.555084,80.026533
3,2024-11-25 05:45:58.153230+00:00,0.135871,0.032920,9.897630,0.000133,-0.000399,0.000133,-46.856251,21.075001,-1.687500,13.555088,80.026540
4,2024-11-25 05:45:48.154593+00:00,0.155025,0.013767,9.888053,0.000000,-0.000266,0.000133,-46.162502,20.775002,-2.250000,13.555085,80.026530
5,2024-11-25 05:45:38.154092+00:00,0.126294,0.032920,9.902418,-0.000266,0.000266,-0.000133,20.006250,26.400002,35.681252,13.555082,80.026537
6,2024-11-25 05:45:28.153812+00:00,0.145448,0.023343,9.911995,-0.000133,0.000266,-0.000133,-32.268753,41.362503,0.187500,13.555084,80.026541
7,2024-11-25 05:45:18.153803+00:00,0.140659,0.018555,9.907207,-0.000399,-0.000266,-0.000533,-32.343750,41.137501,0.000000,13.555079,80.026543
8,2024-11-25 05:45:08.153736+00:00,0.135871,0.023343,9.916783,-0.000399,-0.000266,-0.000533,-32.362503,41.306252,-0.731250,13.555084,80.026538
9,2024-11-25 05:44:58.153316+00:00,0.140659,0.028132,9.916783,-0.000266,-0.000133,-0.000399,-31.950001,40.931252,0.206250,13.555083,80.026536


In [4]:
df = df.iloc[::-1].reset_index(drop=True)

# Ensure the timestamp column is in datetime format
df["timestamp"] = pd.to_datetime(df["timestamp"])

df["dt"] = df["timestamp"].diff().dt.total_seconds()
df["dt"] = df["dt"].fillna(0)

In [5]:
df

,timestamp,ax,ay,az,gx,gy,gz,mx,my,mz,Latitude,Longitude,dt
0,2024-11-25 05:44:58.153316+00:00,0.140659,0.028132,9.916783,-0.000266,-0.000133,-0.000399,-31.950001,40.931252,0.206250,13.555083,80.026536,0.000000
1,2024-11-25 05:45:08.153736+00:00,0.135871,0.023343,9.916783,-0.000399,-0.000266,-0.000533,-32.362503,41.306252,-0.731250,13.555084,80.026538,10.000420
2,2024-11-25 05:45:18.153803+00:00,0.140659,0.018555,9.907207,-0.000399,-0.000266,-0.000533,-32.343750,41.137501,0.000000,13.555079,80.026543,10.000067
3,2024-11-25 05:45:28.153812+00:00,0.145448,0.023343,9.911995,-0.000133,0.000266,-0.000133,-32.268753,41.362503,0.187500,13.555084,80.026541,10.000009
4,2024-11-25 05:45:38.154092+00:00,0.126294,0.032920,9.902418,-0.000266,0.000266,-0.000133,20.006250,26.400002,35.681252,13.555082,80.026537,10.000280
5,2024-11-25 05:45:48.154593+00:00,0.155025,0.013767,9.888053,0.000000,-0.000266,0.000133,-46.162502,20.775002,-2.250000,13.555085,80.026530,10.000501
6,2024-11-25 05:45:58.153230+00:00,0.135871,0.032920,9.897630,0.000133,-0.000399,0.000133,-46.856251,21.075001,-1.687500,13.555088,80.026540,9.998637
7,2024-11-25 05:46:08.153831+00:00,0.145448,0.018555,9.883265,0.000133,-0.000266,0.000133,-53.362503,27.637501,-8.662500,13.555084,80.026533,10.000601
8,2024-11-26 04:53:34.718018+00:00,0.054468,4.452617,8.825027,-0.070041,0.160056,0.026232,-40.143753,15.562501,5.343750,13.556643,80.025030,83246.564187
9,2024-11-27 12:07:22.641368+00:00,0.614711,6.123769,7.642292,0.056992,-0.021838,0.032624,-40.856251,-6.431250,2.212500,13.556612,80.025009,112427.923350


In [6]:
def kalman_filter(state_estimate, estimate_covariance, process_variance, measurement_variance, measurement, control_input, dt):
    predicted_state = state_estimate + control_input * dt
    predicted_covariance = estimate_covariance + process_variance
    kalman_gain = predicted_covariance / (predicted_covariance + measurement_variance)
    updated_state = predicted_state + kalman_gain * (measurement - predicted_state)
    updated_covariance = (1 - kalman_gain) * predicted_covariance

    return updated_state, updated_covariance

import numpy as np

def calculate_lean_angle(df):
    df['pitch'] = np.arctan2(-df['ax'], np.sqrt(df['ay']**2 + df['az']**2)) * (180 / np.pi)
    df['roll'] = np.arctan2(df['ay'], np.sqrt(df['ax']**2 + df['az']**2)) * (180 / np.pi)

    df['gx'] = df['gx'] * (180 / np.pi)
    df['gy'] = df['gy'] * (180 / np.pi)

    pitch_state, pitch_cov = 0.0, 1.0
    roll_state, roll_cov = 0.0, 1.0
    process_variance = 0.01
    measurement_variance = 0.1

    filtered_pitch = []
    filtered_roll = []

    # Kalman filter for pitch and roll fusion
    for i in range(len(df)):
        dt = df['dt'].iloc[i]
        pitch_measurement = df['pitch'].iloc[i]
        roll_measurement = df['roll'].iloc[i]
        gx_control = df['gx'].iloc[i]
        gy_control = df['gy'].iloc[i]

        # Update pitch state
        pitch_state, pitch_cov = kalman_filter(
            state_estimate=pitch_state,
            estimate_covariance=pitch_cov,
            process_variance=process_variance,
            measurement_variance=measurement_variance,
            measurement=pitch_measurement,
            control_input=gx_control,
            dt=dt
        )

        # Update roll state
        roll_state, roll_cov = kalman_filter(
            state_estimate=roll_state,
            estimate_covariance=roll_cov,
            process_variance=process_variance,
            measurement_variance=measurement_variance,
            measurement=roll_measurement,
            control_input=gy_control,
            dt=dt
        )

        filtered_pitch.append(pitch_state)
        filtered_roll.append(roll_state)

    # Append filtered results to the dataframe
    df['filtered_pitch'] = filtered_pitch
    df['filtered_roll'] = filtered_roll   
    
    df['filtered_pitch_rad'] = np.deg2rad(df['filtered_pitch'])  # Convert from degrees to radians
    df['filtered_roll_rad'] = np.deg2rad(df['filtered_roll'])    # Convert from degrees to radians

    df['lean_angle_rad'] = np.arctan2(
        np.sqrt(df['filtered_pitch_rad']**2 + df['filtered_roll_rad']**2), 1
    )
    df['lean_angle_deg'] = np.rad2deg(df['lean_angle_rad'])  

In [7]:
calculate_lean_angle(df)
df

,timestamp,ax,ay,az,gx,gy,gz,mx,my,mz,...,Longitude,dt,pitch,roll,filtered_pitch,filtered_roll,filtered_pitch_rad,filtered_roll_rad,lean_angle_rad,lean_angle_deg
0,2024-11-25 05:44:58.153316+00:00,0.140659,0.028132,9.916783,-0.015259,-0.007629,-0.000399,-31.950001,40.931252,0.206250,...,80.026536,0.000000,-0.812624,0.162520,-0.739414,0.147878,-0.012905,0.002581,0.013160,0.754013
1,2024-11-25 05:45:08.153736+00:00,0.135871,0.023343,9.916783,-0.022888,-0.015259,-0.000533,-32.362503,41.306252,-0.731250,...,80.026538,10.000420,-0.784965,0.134858,-0.876183,0.065415,-0.015292,0.001142,0.015334,0.878553
2,2024-11-25 05:45:18.153803+00:00,0.140659,0.018555,9.907207,-0.022888,-0.015259,-0.000533,-32.343750,41.137501,0.000000,...,80.026543,10.000067,-0.813411,0.107297,-0.995415,-0.014060,-0.017373,-0.000245,0.017373,0.995414
3,2024-11-25 05:45:28.153812+00:00,0.145448,0.023343,9.911995,-0.007629,0.015259,-0.000133,-32.268753,41.362503,0.187500,...,80.026541,10.000009,-0.840691,0.134921,-0.997211,0.137365,-0.017405,0.002397,0.017567,1.006524
4,2024-11-25 05:45:38.154092+00:00,0.126294,0.032920,9.902418,-0.015259,0.015259,-0.000133,20.006250,26.400002,35.681252,...,80.026537,10.000280,-0.730699,0.190462,-1.025329,0.260407,-0.017895,0.004545,0.018461,1.057761
5,2024-11-25 05:45:48.154593+00:00,0.155025,0.013767,9.888053,0.000000,-0.015259,0.000133,-46.162502,20.775002,-2.250000,...,80.026530,10.000501,-0.898207,0.079760,-0.989204,0.099840,-0.017265,0.001743,0.017351,0.994129
6,2024-11-25 05:45:58.153230+00:00,0.135871,0.032920,9.897630,0.007629,-0.022888,0.000133,-46.856251,21.075001,-1.687500,...,80.026540,9.998637,-0.786481,0.190551,-0.877827,-0.040316,-0.015321,-0.000704,0.015336,0.878683
7,2024-11-25 05:46:08.153831+00:00,0.145448,0.018555,9.883265,0.007629,-0.015259,0.000133,-53.362503,27.637501,-8.662500,...,80.026533,10.000601,-0.843135,0.107557,-0.812932,-0.110562,-0.014188,-0.001930,0.014318,0.820360
8,2024-11-26 04:53:34.718018+00:00,0.054468,4.452617,8.825027,-4.013058,9.170525,0.026232,-40.143753,15.562501,5.343750,...,80.025030,83246.564187,-0.315717,26.772526,-243126.832461,555592.080023,-4243.363726,9696.911094,1.570702,89.994587
9,2024-11-27 12:07:22.641368+00:00,0.614711,6.123769,7.642292,3.265378,-1.251220,0.032624,-40.856251,-6.431250,2.212500,...,80.025009,112427.923350,-3.591728,38.615077,90357.220714,302378.061751,1577.031004,5277.492763,1.570615,89.989598


In [9]:
desired_order = ["timestamp", "ax", "ay", "az", "gx", "gy", "gz", "mx", "my", "mz", "Latitude", "Longitude", "dt", "lean_angle_deg"]
df1 = df[desired_order]
df1

,timestamp,ax,ay,az,gx,gy,gz,mx,my,mz,Latitude,Longitude,dt,lean_angle_deg
0,2024-11-25 05:44:58.153316+00:00,0.140659,0.028132,9.916783,-0.015259,-0.007629,-0.000399,-31.950001,40.931252,0.206250,13.555083,80.026536,0.000000,0.754013
1,2024-11-25 05:45:08.153736+00:00,0.135871,0.023343,9.916783,-0.022888,-0.015259,-0.000533,-32.362503,41.306252,-0.731250,13.555084,80.026538,10.000420,0.878553
2,2024-11-25 05:45:18.153803+00:00,0.140659,0.018555,9.907207,-0.022888,-0.015259,-0.000533,-32.343750,41.137501,0.000000,13.555079,80.026543,10.000067,0.995414
3,2024-11-25 05:45:28.153812+00:00,0.145448,0.023343,9.911995,-0.007629,0.015259,-0.000133,-32.268753,41.362503,0.187500,13.555084,80.026541,10.000009,1.006524
4,2024-11-25 05:45:38.154092+00:00,0.126294,0.032920,9.902418,-0.015259,0.015259,-0.000133,20.006250,26.400002,35.681252,13.555082,80.026537,10.000280,1.057761
5,2024-11-25 05:45:48.154593+00:00,0.155025,0.013767,9.888053,0.000000,-0.015259,0.000133,-46.162502,20.775002,-2.250000,13.555085,80.026530,10.000501,0.994129
6,2024-11-25 05:45:58.153230+00:00,0.135871,0.032920,9.897630,0.007629,-0.022888,0.000133,-46.856251,21.075001,-1.687500,13.555088,80.026540,9.998637,0.878683
7,2024-11-25 05:46:08.153831+00:00,0.145448,0.018555,9.883265,0.007629,-0.015259,0.000133,-53.362503,27.637501,-8.662500,13.555084,80.026533,10.000601,0.820360
8,2024-11-26 04:53:34.718018+00:00,0.054468,4.452617,8.825027,-4.013058,9.170525,0.026232,-40.143753,15.562501,5.343750,13.556643,80.025030,83246.564187,89.994587
9,2024-11-27 12:07:22.641368+00:00,0.614711,6.123769,7.642292,3.265378,-1.251220,0.032624,-40.856251,-6.431250,2.212500,13.556612,80.025009,112427.923350,89.989598


In [10]:
import math
import matplotlib.pyplot as plt


In [15]:
def haversine(lat1, lon1, lat2, lon2):
    R = 6371e3  # Earth radius in meters
    phi1 = math.radians(lat1)
    phi2 = math.radians(lat2)
    delta_phi = math.radians(lat2 - lat1)
    delta_lambda = math.radians(lon2 - lon1)

    a = math.sin(delta_phi / 2) ** 2 + math.cos(phi1) * math.cos(phi2) * math.sin(delta_lambda / 2) ** 2
    c = 2 * math.atan2(math.sqrt(a), math.sqrt(1 - a))

    distance = R * c
    return distance

def calculate_speed(row1, row2):
    distance = haversine(row1['Latitude'], row1['Longitude'], row2['Latitude'], row2['Longitude'])
    time_delta = (row2['timestamp'] - row1['timestamp']).total_seconds()
    if time_delta == 0:
        return 0, 0
    speed = distance / time_delta
    return distance, speed

In [16]:
df['cumulative_distance'] = 0
df['cumulative_time'] = 0
df['cumulative_speed'] = 0

In [17]:
# Calculate speeds and cumulative distances
cumulative_distance = 0
cumulative_time = 0

for i in range(1, len(df)):
    distance, speed = calculate_speed(df.iloc[i-1], df.iloc[i])
    df.at[i, 'speed_m_s'] = speed
    cumulative_distance += distance
    cumulative_time += (df.iloc[i]['timestamp'] - df.iloc[i-1]['timestamp']).total_seconds()
    df.at[i, 'cumulative_distance'] = cumulative_distance
    df.at[i, 'cumulative_time'] = cumulative_time
    if cumulative_time != 0:
        df.at[i, 'cumulative_speed'] = cumulative_distance / cumulative_time


C:\Users\praty\AppData\Local\Temp\ipykernel_20040\3686596830.py:10: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '0.2722950649056788' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  df.at[i, 'cumulative_distance'] = cumulative_distance
C:\Users\praty\AppData\Local\Temp\ipykernel_20040\3686596830.py:11: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '10.00042' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  df.at[i, 'cumulative_time'] = cumulative_time
C:\Users\praty\AppData\Local\Temp\ipykernel_20040\3686596830.py:13: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '0.027228362899326106' has dtype incompatible with int64, please explicitly cast to a compatible dtype

In [18]:
df 

,timestamp,ax,ay,az,gx,gy,gz,mx,my,mz,...,filtered_pitch,filtered_roll,filtered_pitch_rad,filtered_roll_rad,lean_angle_rad,lean_angle_deg,cumulative_distance,cumulative_time,cumulative_speed,speed_m_s
0,2024-11-25 05:44:58.153316+00:00,0.140659,0.028132,9.916783,-0.015259,-0.007629,-0.000399,-31.950001,40.931252,0.206250,...,-0.739414,0.147878,-0.012905,0.002581,0.013160,0.754013,0.000000,0.000000,0.000000,NaN
1,2024-11-25 05:45:08.153736+00:00,0.135871,0.023343,9.916783,-0.022888,-0.015259,-0.000533,-32.362503,41.306252,-0.731250,...,-0.876183,0.065415,-0.015292,0.001142,0.015334,0.878553,0.272295,10.000420,0.027228,0.027228
2,2024-11-25 05:45:18.153803+00:00,0.140659,0.018555,9.907207,-0.022888,-0.015259,-0.000533,-32.343750,41.137501,0.000000,...,-0.995415,-0.014060,-0.017373,-0.000245,0.017373,0.995414,0.963444,20.000487,0.048171,0.069114
3,2024-11-25 05:45:28.153812+00:00,0.145448,0.023343,9.911995,-0.007629,0.015259,-0.000133,-32.268753,41.362503,0.187500,...,-0.997211,0.137365,-0.017405,0.002397,0.017567,1.006524,1.597387,30.000496,0.053245,0.063394
4,2024-11-25 05:45:38.154092+00:00,0.126294,0.032920,9.902418,-0.015259,0.015259,-0.000133,20.006250,26.400002,35.681252,...,-1.025329,0.260407,-0.017895,0.004545,0.018461,1.057761,2.191362,40.000776,0.054783,0.059396
5,2024-11-25 05:45:48.154593+00:00,0.155025,0.013767,9.888053,0.000000,-0.015259,0.000133,-46.162502,20.775002,-2.250000,...,-0.989204,0.099840,-0.017265,0.001743,0.017351,0.994129,3.037235,50.001277,0.060743,0.084583
6,2024-11-25 05:45:58.153230+00:00,0.135871,0.032920,9.897630,0.007629,-0.022888,0.000133,-46.856251,21.075001,-1.687500,...,-0.877827,-0.040316,-0.015321,-0.000704,0.015336,0.878683,4.198323,59.999914,0.069972,0.116125
7,2024-11-25 05:46:08.153831+00:00,0.145448,0.018555,9.883265,0.007629,-0.015259,0.000133,-53.362503,27.637501,-8.662500,...,-0.812932,-0.110562,-0.014188,-0.001930,0.014318,0.820360,5.126803,70.000515,0.073240,0.092842
8,2024-11-26 04:53:34.718018+00:00,0.054468,4.452617,8.825027,-4.013058,9.170525,0.026232,-40.143753,15.562501,5.343750,...,-243126.832461,555592.080023,-4243.363726,9696.911094,1.570702,89.994587,242.692758,83316.564702,0.002913,0.002854
9,2024-11-27 12:07:22.641368+00:00,0.614711,6.123769,7.642292,3.265378,-1.251220,0.032624,-40.856251,-6.431250,2.212500,...,90357.220714,302378.061751,1577.031004,5277.492763,1.570615,89.989598,246.853941,195744.488052,0.001261,0.000037


In [19]:
desired_order = ["timestamp", "ax", "ay", "az", "gx", "gy", "gz", "mx", "my", "mz", "Latitude", "Longitude", "dt", "lean_angle_deg", "speed_m_s"]
df1 = df[desired_order]
df1

,timestamp,ax,ay,az,gx,gy,gz,mx,my,mz,Latitude,Longitude,dt,lean_angle_deg,speed_m_s
0,2024-11-25 05:44:58.153316+00:00,0.140659,0.028132,9.916783,-0.015259,-0.007629,-0.000399,-31.950001,40.931252,0.206250,13.555083,80.026536,0.000000,0.754013,NaN
1,2024-11-25 05:45:08.153736+00:00,0.135871,0.023343,9.916783,-0.022888,-0.015259,-0.000533,-32.362503,41.306252,-0.731250,13.555084,80.026538,10.000420,0.878553,0.027228
2,2024-11-25 05:45:18.153803+00:00,0.140659,0.018555,9.907207,-0.022888,-0.015259,-0.000533,-32.343750,41.137501,0.000000,13.555079,80.026543,10.000067,0.995414,0.069114
3,2024-11-25 05:45:28.153812+00:00,0.145448,0.023343,9.911995,-0.007629,0.015259,-0.000133,-32.268753,41.362503,0.187500,13.555084,80.026541,10.000009,1.006524,0.063394
4,2024-11-25 05:45:38.154092+00:00,0.126294,0.032920,9.902418,-0.015259,0.015259,-0.000133,20.006250,26.400002,35.681252,13.555082,80.026537,10.000280,1.057761,0.059396
5,2024-11-25 05:45:48.154593+00:00,0.155025,0.013767,9.888053,0.000000,-0.015259,0.000133,-46.162502,20.775002,-2.250000,13.555085,80.026530,10.000501,0.994129,0.084583
6,2024-11-25 05:45:58.153230+00:00,0.135871,0.032920,9.897630,0.007629,-0.022888,0.000133,-46.856251,21.075001,-1.687500,13.555088,80.026540,9.998637,0.878683,0.116125
7,2024-11-25 05:46:08.153831+00:00,0.145448,0.018555,9.883265,0.007629,-0.015259,0.000133,-53.362503,27.637501,-8.662500,13.555084,80.026533,10.000601,0.820360,0.092842
8,2024-11-26 04:53:34.718018+00:00,0.054468,4.452617,8.825027,-4.013058,9.170525,0.026232,-40.143753,15.562501,5.343750,13.556643,80.025030,83246.564187,89.994587,0.002854
9,2024-11-27 12:07:22.641368+00:00,0.614711,6.123769,7.642292,3.265378,-1.251220,0.032624,-40.856251,-6.431250,2.212500,13.556612,80.025009,112427.923350,89.989598,0.000037
